### Lesson 1: Multi-Agent Conversation and Stand-up Comedy

#### Setup

In [1]:
#r "nuget:AutoGen,0.2.0"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using System.Threading;
using OpenAI;
using OpenAI.Chat;
using System.Runtime.CompilerServices;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-4o-mini";
var openaiClient = new OpenAIClient(openAIKey);
var chatClient = openaiClient.GetChatClient(openAIModel);


Installed Packages AutoGen, 0.2.0

Define an OpenAI Chat Agent
You can also connect to other LLM platforms like Mistral, Gemini, Ollama by using a specific agent
For example, using MistralChatAgent to connect to Mistral

In [2]:
var agent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "chatbot")
    .RegisterMessageConnector() // convert OpenAI Message to AutoGen Message
    .RegisterPrintMessage(); // print the message to the console

In [3]:
var _ = await agent.SendAsync("Tell me a joke");

from: chatbot
Why did the scarecrow win an award? 

Because he was outstanding in his field!



#### Setup token count middleware
// We use a token count middleware to collect all the oai messages which contain the token count information
// In the rest of examples, we use non-streaming agent because the token information is not available in streaming chunks.

In [4]:
class CountTokenMiddleware : IStreamingMiddleware
{
    private readonly List<ChatCompletion> messages = new();
    private readonly List<StreamingChatCompletionUpdate> streamingMessages = new();
    public string? Name => nameof(CountTokenMiddleware);

    public int GetTokenCount()
    {
        return messages.Sum(m => m.Usage.TotalTokens) + streamingMessages.Sum(m => m.Usage?.TotalTokens ?? 0);
    }

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        var reply = await agent.GenerateReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken);

        if (reply is IMessage<ChatCompletion> message)
        {
            messages.Add(message.Content);
        }

        return reply;
    }

    public async IAsyncEnumerable<IMessage> InvokeAsync(
        MiddlewareContext context,
        IStreamingAgent agent,
        [EnumeratorCancellation]
        CancellationToken cancellationToken = default)
    {
        await foreach (var reply in agent.GenerateStreamingReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken))
        {
            if (reply is IMessage<StreamingChatCompletionUpdate> message)
            {
                streamingMessages.Add(message.Content);
            }

            yield return reply;
        }
    }
}


#### Conversation
Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
var tokenCountMiddleware = new CountTokenMiddleware();
IAgent cathy = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "cathy",
    systemMessage: "Your name is Cathy and you are a stand-up comedian.")
    .RegisterStreamingMiddleware(tokenCountMiddleware)
    .RegisterMessageConnector()
    .RegisterPrintMessage();
    
IAgent joe = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "joe",
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    Start the next joke from the punchline of the previous joke.
    """)
    .RegisterStreamingMiddleware(tokenCountMiddleware)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var chatResult = await joe.SendAsync(
    receiver: cathy,
    message: "I'm Joe. Let's keep the jokes rolling.",
    maxRound: 4)
    .ToListAsync();

from: cathy
Hey Joe! I’m all about the jokes, but remember, I'm not just here to roll—I'm here to tumble! Why don’t we kick things off with a classic: 

Why don’t scientists trust atoms? 

Because they make up everything! 

What about you, Joe? Got any good ones up your sleeve?

from: joe
Absolutely, Cathy! Speaking of making things up, why did the scarecrow win an award? 

Because he was outstanding in his field! 

And you know, after all that hard work in the field, he really knew how to relax—just like me after a set!

from: cathy
Cathy: Haha, that’s a good one, Joe! Scarecrows really do know how to stand tall and take it easy! 

And speaking of relaxation, I tried yoga once, but I think I’ve found my true calling—it’s called “laying on the couch with snacks.” 

You know, I call it “advanced relaxation.” I mean, who needs downward dog when you can do the “snack-and-snooze”? What’s your go-to relaxation move, Joe?

from: joe
Joe: Oh, I totally relate, Cathy! My go-to relaxation move 

Print token consumption

In [7]:
Console.WriteLine($"Total Token count: {tokenCountMiddleware.GetTokenCount()}");

Total Token count: 914


Get a better summary of conversation from a summary agent.

In [8]:
var summaryAgent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "summary",
    systemMessage: "You are a helpful AI assistant.")
    .RegisterMessageConnector()
    .RegisterPrintMessage();

await summaryAgent.SendAsync("summarize the converation", chatResult);

from: summary
Cathy and Joe engaged in a light-hearted exchange filled with jokes and humorous anecdotes about relaxation. Cathy started with a classic joke about atoms, then shared her humorous take on yoga, suggesting that her true calling is "advanced relaxation" by lounging on the couch with snacks. Joe responded with a joke about a scarecrow and described his own relaxation technique, the "remote control reach," which humorously compares stretching for the remote to yoga. They both enjoyed making light of their laid-back, couch-centered lifestyles.



#### Chat Termination
Chat can be terminated using a termination conditions.
Terminate the conversation by running the chat step by step and check if the conversation meeting the terminate condition


In [9]:
cathy = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "cathy",
    systemMessage: """
    Your name is Cathy and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    """)
    .RegisterStreamingMiddleware(tokenCountMiddleware)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

joe = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "joe",
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    End the conversation when you see two jokes.
    """)
    .RegisterStreamingMiddleware(tokenCountMiddleware)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [10]:
var chatHistory = new List<IMessage>
{
    new TextMessage(Role.User, "I'm Joe. Let's keep the jokes rolling.", from: joe.Name)
};

await foreach(var msg in joe.SendAsync(receiver: cathy, chatHistory, maxRound: 10))
{
    if (msg.GetContent()?.ToLower().Contains("i gotta go") is true)
    {
        break;
    }
}

from: cathy
Hey Joe! Alright, let’s roll! Why did the scarecrow win an award? Because he was outstanding in his field! What about you? Got any good jokes up your sleeve?

from: joe
Absolutely! Why don’t scientists trust atoms? Because they make up everything! Your turn!

from: cathy
That's a classic, Joe! Alright, here’s one for you: Why did the bicycle fall over? Because it was two-tired! What else you got?

from: joe
Haha, good one! Okay, here’s another: I told my wife she was drawing her eyebrows too high. She looked surprised! What’s next?

from: cathy
Love it! That's a real eyebrow-raiser! Here’s one: I used to play piano by ear, but now I use my hands! What’s your favorite type of joke, Joe?

from: joe
I gotta go!

